# Decorator class

so far we are using the functional approach to decorator the class or function to decorate it.

## Functional Approach

In [29]:
#? decorating the function
import inspect
import types
from functools import wraps

def func_decorator(fn):
    @wraps(fn)
    def inner(*arg,**kwargs):
        print(f"decorating the function {fn.__qualname__}")
        return fn(*arg,**kwargs)
    return inner


In [13]:
@func_decorator
def hello():
    return "say hello"

In [14]:
hello()

decorating the function hello


'say hello'

In [16]:
#? decorating the class
def class_decorator(cls):
    print(f"decorating the cls : {cls.__qualname__}")
    for name,obj in vars(cls).items():
        if callable(obj):
            setattr(cls,name,func_decorator(obj))
    return cls

In [17]:
@class_decorator
class Person:
    def __init__(self,name):
        self.name = name
    def greet(self):
        return f"{self.name} say hello"

decorating the cls : Person


In [18]:
p = Person("python")

decorating the function Person.__init__


In [19]:
p.greet()

decorating the function Person.greet


'python say hello'

so far we are using the function of decorate the function or class.
Similar we can use the class to decorate  function or class

In [26]:
#? decorating the function
class Logger:
    def __init__(self,fn):
        self.fn = fn
    def __call__(self, *args, **kwargs):
        print(f"Logging : {self.fn.__qualname__} function")
        return self.fn(*args,**kwargs)

In [27]:
@Logger
def hello(name):
    print(f" {name} say hello")

In [28]:
hello("python")

Logging : hello function
 python say hello


When we are using class to decorate the function it's okay.

but we use the class to decorate the method this make problem .we need to be careful.

In [30]:
class Person:
    def __init__(self,name):
        self.name = name
    @Logger
    def greet(self):
        return f"{self.name} say hello"

In [31]:
p = Person("python")

In [37]:
try:
    p.greet()
except TypeError as ex:
    print(ex)

Logging : Person.greet function
Person.greet() missing 1 required positional argument: 'self'


What's going on here? Why is Python complaining that self has not been passed to say_hello?

We called it from an instance, so why is self not being passed to it.

Well, you have to remember what say_hello is now that it has been decorated - it is an instance of a class, not a function!

And do you remember how functions are turned into methods?

The descriptor protocol... Functions implement a __get__ method, and that is ultimately used to create the bound method.

Our class does not implement the __get__ method, so that callable remain a plain callable, not a bound method, and that's why our implementation is broken.

Why typeerror is raised?
1. when we use the `class decorator` to decorate the function or method it returns` instance of class decorator`
2. since we implement the` __call__` method , instance are callable.
3. Instance of class decorator take argument and call the original function.
4. this work well in normal function. but when we are working with method, these are bound to some object.
5. method are expecting the bounded object as first argument.

In [38]:
class Person:
    def __init__(self,name):
        self.name = name
    @Logger
    def greet(self):
        return f"{self.name} say hello"

In [39]:
Person.greet
#! instance of the class Logger

# How to overcome this problem?


method are non-data descriptor.

In [49]:
class Person:
    def __init__(self,name):
        self.name = name
    def greet(self):
        return f"{self.name} say hello"

In [71]:
hasattr(Person.__dict__["greet"],"__get__") ,hasattr(Person.__dict__["greet"],"__set__")
#? non data description

(True, False)

so we can implement descriptor protocol for the class Descriptor

In [80]:
class Logger:
    def __init__(self,fn):
        self.fn = fn
    def __call__(self, *args, **kwargs):
        print(f"Logging : {self.fn.__qualname__} function")
        return self.fn(*args,**kwargs)
    def __get__(self, instance, owner):
        if instance is None:
            return self
        return types.MethodType(self.fn,instance)

In [81]:
class Person:
    def __init__(self,name):
        self.name = name
    @Logger
    def greet(self):
        return f"{self.name} say hello"

In [82]:
p = Person("python")

In [84]:
p.greet()

'python say hello'